In [1]:
import os
import csv
import uuid
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

from groq import Groq
load_dotenv()

In [2]:
# Embedding model
encoder = SentenceTransformer("all-MiniLM-L6-v2")

# Qdrant (LOCAL / Docker)
qdrant = QdrantClient(":memory:")

# LLM

llm = Groq(api_key=os.getenv("GROQ_API_KEY"))


In [6]:
# =========================
# CONVERSATION MEMORY
# =========================

USER_MEMORY = []
MAX_MEMORY = 5

def store_user_memory(query: str):
    query = query.strip().lower()

    # 🔹 Avoid consecutive duplicates
    if USER_MEMORY and USER_MEMORY[-1] == query:
        return

    USER_MEMORY.append(query)

    # 🔥 Remove OLDEST (from front)
    if len(USER_MEMORY) > MAX_MEMORY:
        USER_MEMORY.pop(0)


def get_user_memory():
    # 🔁 Return MOST RECENT FIRST
    return USER_MEMORY[::-1]


In [7]:
def is_specific_compound_query(query: str) -> bool:
    keywords = ["explain", "what is", "describe", "information about"]
    return any(k in query.lower() for k in keywords)


In [8]:
def retrieve_images(query, top_k=5):
    hits = qdrant.query_points(
        IMAGE_COLLECTION,
        query=encoder.encode(query).tolist(),
        limit=top_k * 3,  # overfetch
        with_payload=True
    ).points

    seen = {}
    for h in hits:
        name = h.payload["compound_name"].lower().strip()

        if name not in seen or h.score > seen[name]["score"]:
            seen[name] = {
                "compound": name,
                "image": h.payload["image"],
                "score": round(h.score, 3)
            }

    return list(seen.values())[:top_k]


In [9]:
def filter_image_text_intersection(images):
    valid = []

    for img in images:
        compound = img["compound"]

        text_hits = qdrant.query_points(
            collection_name=TEXT_COLLECTION,
            query=encoder.encode(compound).tolist(),
            limit=1,
            with_payload=True
        ).points

        if text_hits:
            valid.append({
                "compound": compound,
                "image": img["image"],
                "image_score": img["score"],
                "text": text_hits[0].payload["features"],
                "text_score": text_hits[0].score
            })

    return valid


In [64]:
def choose_best_compound(valid_compounds):
    return max(
        valid_compounds,
        key=lambda x: (x["image_score"] + x["text_score"])
    )


In [66]:
def recommend_related_compounds(compound, top_k=3):
    compound = compound.lower().strip()

    hits = qdrant.query_points(
        collection_name=TEXT_COLLECTION,
        query=encoder.encode(compound).tolist(),
        limit=top_k + 3,   # overfetch
        with_payload=True
    ).points

    seen = set()
    recommendations = []

    for h in hits:
        name = h.payload["compound_name"].lower().strip()

        if name == compound:
            continue

        if name not in seen:
            seen.add(name)
            recommendations.append(name)

        if len(recommendations) == top_k:
            break

    return recommendations


In [68]:
def encode_text(text):
    return encoder.encode(text).tolist()

In [70]:
def search_by_text(query, memory, top_k=3):

    # 🔥 MEMORY KO SEARCH MEIN MAT GHUSAO
    query_vector = encoder.encode(query).tolist()

    response = qdrant.query_points(
        collection_name=TEXT_COLLECTION,
        query=query_vector,
        limit=top_k,
        with_payload=True
    )

    results = []
    for hit in response.points:
        results.append({
            "compound": hit.payload["compound_name"],
            "text": hit.payload["features"],
            "score": round(hit.score, 3)
        })

    return results


In [72]:
def rag_answer(query, chosen, memory):
    prompt = f"""
You are a chemistry assistant.

Answer ONLY using the information below.

Compound: {chosen['compound']}
Description:
{chosen['text']}

User History:
{', '.join(memory)}

Question:
{query}
"""

    response = llm.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content
